In [2]:
import pickle as pkl
from datetime import datetime
from collections import defaultdict
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression

from tqdm import tqdm

In [2]:
tournaments = pkl.load(open('tournaments.pkl', 'rb'))
results = pkl.load(open('results.pkl', 'rb'))
players = pkl.load(open('players.pkl', 'rb'))

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
 
*   взять в тренировочный набор турниры с dateStart из 2019 года;
*   в тестовый — турниры с dateStart из 2020 года.

In [4]:
train = {
    'player_id': [],
    'player_name': [],
    'player_rating': [],
    
    'team_id': [],
    'team_position': [],
    
    'tournament_id': [],
    'tournament_name': [],
    'question_order': [],
    'is_answered': []
}
test = {
    'player_id': [],
    'player_name': [],
    'player_rating': [],
    
    'team_id': [],
    'team_position': [],
    
    'tournament_id': [],
    'tournament_name': [],
    'question_order': [],
    'is_answered': []
}

In [5]:
for key, tournament in tqdm(tournaments.items(), position=0, leave=True):
    not_defined_idxs = []
    if datetime.fromisoformat(tournament['dateStart']).year not in [2019, 2020]:
        continue
    # Определим номера вопросов, где хотя бы у одной команды была метка '?' или 'X'
    for team_result in results[key]:
        if 'mask' in team_result.keys():
            if team_result['mask']:
                for i, is_answered in enumerate(team_result['mask']):
                    if is_answered in ['X', '?']:
                        not_defined_idxs.append(i)
    not_defined_idxs = np.unique(np.array(not_defined_idxs))

    for team_result in results[key]:
        for player in team_result['teamMembers']:
            if 'mask' in team_result.keys():
                if team_result['mask']:
                    for i, is_answered in enumerate(team_result['mask']):
                        if i not in not_defined_idxs:
                            if datetime.fromisoformat(tournament['dateStart']).year == 2019:
                                train['player_id'].append(player['player']['id'])
                                train['player_name'].append(f"{player['player']['surname']} {player['player']['name']} {player['player']['patronymic']}")
                                train['player_rating'].append(player['rating'])

                                train['team_id'].append(team_result['team']['id'])
                                train['team_position'].append(team_result['position'])

                                train['tournament_id'].append(key)
                                train['tournament_name'].append(tournament['name'])

                                train['question_order'].append(i)
                                train['is_answered'].append(int(team_result['mask'][i]))
                                
                            elif datetime.fromisoformat(tournament['dateStart']).year == 2020:
                                test['player_id'].append(player['player']['id'])
                                test['player_name'].append(f"{player['player']['surname']} {player['player']['name']} {player['player']['patronymic']}")
                                test['player_rating'].append(player['rating'])

                                test['team_id'].append(team_result['team']['id'])
                                test['team_position'].append(team_result['position'])

                                test['tournament_id'].append(key)
                                test['tournament_name'].append(tournament['name'])

                                test['question_order'].append(i)
                                test['is_answered'].append(int(team_result['mask'][i]))

100%|██████████████████████████████████████████████████████████████████████████████| 5528/5528 [02:30<00:00, 36.71it/s]


In [6]:
train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)

In [7]:
# train_df.to_csv('train.csv', index=False)
# test_df.to_csv('test.csv', index=False)

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [4]:
train_df

,player_id,player_name,player_rating,team_id,team_position,tournament_id,tournament_name,question_order,is_answered
0,6212,Выменец Юрий Яковлевич,13507,45556,1.0,4772,Синхрон северных стран. Зимний выпуск,0,1
1,6212,Выменец Юрий Яковлевич,13507,45556,1.0,4772,Синхрон северных стран. Зимний выпуск,1,1
2,6212,Выменец Юрий Яковлевич,13507,45556,1.0,4772,Синхрон северных стран. Зимний выпуск,2,1
3,6212,Выменец Юрий Яковлевич,13507,45556,1.0,4772,Синхрон северных стран. Зимний выпуск,3,1
4,6212,Выменец Юрий Яковлевич,13507,45556,1.0,4772,Синхрон северных стран. Зимний выпуск,4,1
...,...,...,...,...,...,...,...,...,...
20844793,217156,Рябкова Юлия Александровна,0,76130,2019.0,6255,ОВСЧ,31,0
20844794,217156,Рябкова Юлия Александровна,0,76130,2019.0,6255,ОВСЧ,32,0
20844795,217156,Рябкова Юлия Александровна,0,76130,2019.0,6255,ОВСЧ,33,0
20844796,217156,Рябкова Юлия Александровна,0,76130,2019.0,6255,ОВСЧ,34,0


## 2

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
* повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
* в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
* для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

Обучим модель логистической регрессии на следующих атрибутах:
* доля взятых игроком вопросов за все время от общего числа вопросов по всем отыгранным турнирам - для неявного учета в модели силы игрока;
* доля игроков, взявших вопрос в текущем турнире об общего числа вопросов в турнире - для неявного учета сложности вопроса в рамках турнира.

In [5]:
train_par = train_df[['player_id', 'is_answered']]\
                    .groupby('player_id')\
                    .agg('mean').rename(columns={'is_answered': 'player_answer_rate'})
train_pac = train_df[['player_id', 'is_answered']]\
                    .groupby('player_id')\
                    .agg('count').rename(columns={'is_answered': 'player_answer_count'})
train_tar = train_df[['tournament_id', 'is_answered']]\
                    .groupby('tournament_id')\
                    .agg('mean').rename(columns={'is_answered': 'tournament_answer_rate'})

train_df = train_df.merge(train_par, how='left', left_on='player_id', right_index=True)\
                   .merge(train_pac, how='left', left_on='player_id', right_index=True)\
                   .merge(train_tar, how='left', left_on='tournament_id', right_index=True)\

In [21]:
X_train = train_df[['player_answer_rate', 'player_answer_count', 'tournament_answer_rate']]
y_train = train_df['is_answered']

In [25]:
%%time
model = LogisticRegression(n_jobs=-1)
model.fit(X_train, y_train)

Wall time: 2min 19s


LogisticRegression(n_jobs=-1)

In [36]:
model.coef_, model.intercept_

(array([[ 4.78556920e+00, -1.92493700e-05,  1.79339718e+00]]),
 array([-3.12901435]))

In [27]:
train_df['predict'] = model.predict_proba(train_df[['player_answer_rate', 'player_answer_count', 
                                                    'tournament_answer_rate']])[:, 1]

In [40]:
train_df[['player_id', 'player_name', 'predict']]\
                    .groupby(['player_name', 'player_id'])\
                    .agg('sum').sort_values('predict', ascending=False).head(20)

,,predict
player_name,player_id,
Дворянчиков Алексей Ярославович,110920,5668.982364
Киселёв Иван Сергеевич,131908,5029.616179
Рыжанова Наталия Владимировна,27499,5002.694017
Салихов Максим Юрьевич,147450,4904.776574
Ильин Константин Евгеньевич,12511,4838.815325
Иванченко Сергей Леонидович,12313,4833.967150
Мельникова Ольга Андреевна,20640,4814.839127
Черкасов Сергей Евгеньевич,117262,4570.394003
Раскумандрин Вадим Игоревич,26640,4511.298742


In [39]:
train_df[['player_name', 'player_rating']].groupby('player_name')\
                                          .agg('max').sort_values('player_rating', ascending=False).head(20)

,player_rating
player_name,
Сорожкин Артём Сергеевич,14897
Семушин Иван Николаевич,14789
Савченков Михаил Владимирович,14726
Спешков Сергей Леонидович,14708
Брутер Александра Владимировна,14543
Руссо Максим Михайлович,14543
Либер Александр Витальевич,14464
Островский Андрей Вячеславович,14276
Галачян Ким Гагикович,14239
